## Load Lib

In [178]:
import numpy as np

## Create the classes


In [179]:
class Component:
  def __init__(self,t,n1,n2,v,iv):
    self.type= t
    self.node1=n1
    self.node2=n2
    self.value = v
    self.InitialValue=iv
    self.number = 0
  def __str__(self):
    return self.type + " " + str(self.node1) + " " + str(self.node2) + " " + str(self.value) + " " + str(self.InitialValue)

In [180]:
def addComponent(line):
    return Component(line[0],line[1],line[2],float(line[3]),float(line[4]))

In [181]:
class Curcit:
  def __init__(self):
    self.RComponents = []
    self.VComponents = []
    self.IComponents = []
    self.CComponents = []
    self.LComponents = []
    self.G = None
    self.A = None
    self.B = None
    self.C = None
    self.D = None
    self.Z = None
    self.X = None
    self.ListOfXs = None
    self.timeStep = None
    self.loops=0
    self.M = 0
    self.N=0
  def read_file(self,file_name):
    """
    add a component to the circuit

    """
    with open(file_name) as f:
      lines = [line.strip() for line in f.readlines()]
      self.timeStep = float(lines[0])
      self.loops = int(lines[1])
      # add all commponents to the circuit except the first two lines and last line
      components = [addComponent(line.split()) for line in lines[2:-1]]
      #print the components
      i = 0
      for component in components:
        if component.type == "Vsrc" or component.type == "I":
          component.number = i
          i += 1
        if component.type == "R":
          self.RComponents.append(component)
        elif component.type == "Vsrc":
          self.VComponents.append(component)
        elif component.type == "Isrc":
          self.IComponents.append(component)
        elif component.type == "C":
          self.CComponents.append(component)
        elif component.type == "I":
          self.LComponents.append(component)
      self.M = len(self.LComponents) + len(self.VComponents)
      def num_nodes(self):
        """
        return the number of nodes in the circuit
        by loop though all the nodes and check if the node starts with V 
        """
        nodes = set()
        for component in components:
          if "V" in component.node1:
            nodes.add(component.node1)
          if "V" in component.node2:
            nodes.add(component.node2)
        self.N = len(nodes)-1
      num_nodes(self)
    return
          
          
  def build_G(self):
    """
    build the G matrix
    """
    self.G = np.zeros((self.N,self.N))
    for R in self.RComponents:
      IndexNode1 = int(R.node1[1])-1
      IndexNode2 = int(R.node2[1])-1
      if IndexNode2 != -1:
        self.G[IndexNode2][IndexNode2] += 1/R.value
        self.G[IndexNode1][IndexNode2] -= 1/R.value
        self.G[IndexNode2][IndexNode1] -= 1/R.value
      self.G[IndexNode1][IndexNode1] += 1/R.value
      

    for C in self.CComponents:
      IndexNode1 = int(C.node1[1])-1
      IndexNode2 = int(C.node2[1])-1
      self.G[IndexNode1][IndexNode1] += C.value/self.timeStep
      if IndexNode2 !=-1:
        self.G[IndexNode2][IndexNode2] += C.value/self.timeStep
        self.G[IndexNode1][IndexNode2] -= C.value/self.timeStep
        self.G[IndexNode2][IndexNode1] -= C.value/self.timeStep
    return
  
  def __str__(self):
    return "G: " + str(self.G) + "\n" + "B: " + str(self.B) + "\n" + "C: " + str(self.C) + "\n" + "D: " + str(self.D) + "\n" + "Z: " + str(self.Z) + "\n" + "X: " + str(self.X) + "\n" + "A: " + str(self.A) + "\n"
  
  def build_B(self):
    """
    build the B matrix
    """

    self.B = np.zeros((self.N,self.M))
    for L in self.LComponents:
      IndxNode1 = int(L.node1[1])-1
      IndxNode2 = int(L.node2[1])-1
      pos = L.number
      self.B[IndxNode1][pos] = 1
      if IndxNode2 != -1:
        self.B[IndxNode2][pos] = -1

    for V in self.VComponents:
      IndxNode1 = int(V.node1[1])-1
      IndxNode2 = int(V.node2[1])-1
      pos = V.number
      self.B[IndxNode1][pos] = 1
      if IndxNode2 != -1:
        self.B[IndxNode2][pos] = -1

  def build_C(self):
    """
    build the C matrix
    """
    self.C = np.zeros((self.M,self.N))
    # transposing the B matrix
    self.C = self.B.transpose()

  def build_D(self):
    """
    build the D matrix 
    """
    self.D = np.zeros((self.M,self.M))
    for L in self.LComponents:
      pos = L.number
      self.D[pos][pos] -= L.value/self.timeStep
    return

  def build_Z(self):
    """
    build the Z matrix
    The i matrix is nx1 and contains the sum of the currents through the
    passive elements into the corresponding node (either zero, or the sum
    of independent current sources).

    The e matrix is mx1 and holds the values of the independent voltage
    sources.
    Z = [i;e]
    """
    i = np.zeros((self.N,1))
    E = np.zeros((self.M,1))
    for V in self.VComponents:
      pos = V.number
      E[pos] = V.value
    for I in self.IComponents:
      IndxNode1 = int(I.node1[1])-1
      IndxNode2 = int(I.node2[1])-1
      if IndxNode2 != -1:
        i[IndxNode2] -= float(I.value)
      i[IndxNode1] += float(I.value)

    # add the Capacitor current
    for C in self.CComponents:
      IndxNode1 = int(C.node1[1])-1
      IndxNode2 = int(C.node2[1])-1
      # the current through the capacitor is the voltage across the capacitor devided by the time step
      V = self.X[IndxNode1] - self.X[IndxNode2] if IndxNode2 != -1 else self.X[IndxNode1]
      i[IndxNode1] += C.value*V/self.timeStep
      if IndxNode2 != -1:
        i[IndxNode2] -= C.value*V/self.timeStep
      

    # add the inductor current
    idx = 0 
    for L in self.LComponents:
      IndxNode1 = int(L.node1[1])-1
      IndxNode2 = int(L.node2[1])-1
      pos = L.number
      I_V = self.X[pos+self.N] # the current through the inductor is the voltage across the inductor devided by the time step
      E[idx] -= L.value*I_V/self.timeStep # we add the current to the node if the current is going into the node
      idx += 1


    self.Z = np.block([[i],[E]])
    return
  
  def build_A(self):
    """
    build the A matrix
    """
    self.A = np.block([[self.G,self.B],[self.C,self.D]])
    return
  
  def build_matrices(self):
    """
    build all the matrices
    """
    self.build_G()
    self.build_B()
    self.build_C()
    self.build_D()
    self.build_A()
    return
    
    
  def print(self):
    """
    print the circuit
    """
    print("G:")
    print(self.G)
    print("B:")
    print(self.B)
    print("C:")
    print(self.C)
    print("D:")
    print(self.D)
    print("Z:")
    print(self.Z)
    print("X:")
    print(self.X)
    print("A:")
    print(self.A)
    return
  
  def solve(self):
    """
    solve the circuit
    """
    self.ListOfXs = []
    self.X = np.zeros((self.N+self.M,1))
    #print("N and M: ",self.N,self.M)
    self.build_matrices()
    for i in range(self.loops):
      self.build_Z()
      #print the dimension of the matrices
      self.X = np.linalg.solve(self.A,self.Z)
      if i == self.loops-1:
        print("X: ",self.X)

      self.ListOfXs.append(self.X)
  


In [182]:
# test the circuit

"""
the test case files are 
1.txt
2.txt
3.txt
4.txt
5.txt
6.txt
7.txt
8.txt
"""
testCaesFile = ["1.txt","2.txt","3.txt","4.txt","5.txt","6.txt","7.txt","8.txt"]
for file in testCaesFile:
  circuit = Curcit()
  circuit.read_file(file)
  circuit.solve()


X:  [[10.        ]
 [ 5.        ]
 [-1.66666667]]
X:  [[10.   ]
 [ 1.2  ]
 [-0.044]]
X:  [[-8. ]
 [24. ]
 [20. ]
 [-0.4]
 [ 0.1]]
X:  [[ 2.]
 [ 0.]
 [-4.]]
X:  [[ 1.50000000e+01]
 [ 1.49954891e+01]
 [-4.51092990e-07]]
X:  [[ 8.99729344]
 [ 5.99819563]
 [-2.99909781]
 [ 2.99909781]]
X:  [[0.36533261]
 [0.22344038]]
X:  [[0.1328971 ]
 [0.10003651]
 [1.03286059]]
